## Parser for .res files

Directory should look like this:
```
resluts
| plots.ipynb
└─final_json
   └ json files created by this notebook
└─final_res
   └ res file outputs from server testing
     should have distinct names
└─plots
   └ pdf output of plots
└─presentation
   └ complete presentation
```

In [ ]:
import json
import glob

graph_data = {}
with open('Graphdaten.txt', 'r') as f:
    for line in f.readlines():
        split = line.split()
        graph_data[split[0]] = {'n':split[1], 'm': split[2]}

for file in glob.glob('./final_res/*.res'):
    added_lines = []

    with open(file, 'r') as f:
        lines = f.readlines()
        data = {}
        for line in lines:

            temp = {}

            if 'correct' in line:
                split_arr = line.split('\t')
                time = split_arr[1].strip().split(' ')[0]
                temp['t'] = float(time[time.index(':')+1:]) + 60*float(time[:time.index(':')])
                temp['k'] = int(split_arr[2][1:split_arr[2].index(',')])
            else:
                continue

            if 'complex' in line:
                temp['c'] = 'complex'
            elif 'synthetic' in line:
                temp['c'] = 'synthetic'
            else:
                temp['c'] = 'unknown'

            name = line[line.index(temp['c'] + '/')+len(temp['c'])+1: line.index(' ')]

            if name in added_lines:
                continue
            added_lines.append(name)

            data[name] = temp

    with open('./final_json/' + file[12:] + '.json', 'w') as f:
        json.dump(data, f)

In [ ]:
# Reading in data and creating a dataframe
import json
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import pandas as pd
from math import log
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
from matplotlib.pyplot import figure
figure(figsize=(8, 8), dpi=80);

In [ ]:
# Merging all output json files in to one pandas DataFrame

data = {}
max_time = 180

for json_file in glob.glob('./final_json/*.json'):
    with open(json_file, 'r') as f:
        data[json_file[13:json_file.index('.res')]] = json.loads(f.read())

complete = {}
for dic in data:
    for i in data[dic]:
        if i in complete:
            continue
        else:
            complete[i] = {'k':data[dic][i]['k'],'n':data[dic][i]['n'],'m':data[dic][i]['m'],'c':data[dic][i]['c']}
            for j in data:
                if i in data[j]:
                    complete[i][j] = data[j][i]['t']
                else:
                    complete[i][j] = max_time

df = pd.DataFrame(complete).transpose()

In [ ]:
# completet dataframe
df

In [ ]:
# Comparison of tow models

fig, ax = plt.subplots()

# scatter(<Model y>, <Model x>)
ax.scatter(df['old_dfvs.jar'], df['dfvs_busy_cycle_no_clean'], marker = "x")
ax.set_yscale('log')
ax.set_xlim(0.07, max_time*1.2)
ax.set_ylim(0.07, max_time*1.2)
ax.set_xscale('log')
ax.add_line(mlines.Line2D([0,max_time*1.2], [0,max_time*1.2], color='red', ls='--', lw=1))

# 10x intervalls
ax.add_line(mlines.Line2D([0.01,100], [0.1,1000], color='green', ls='--', lw=0.5))
ax.add_line(mlines.Line2D([0.1,1000], [0.01,100], color='green', ls='--', lw=0.5))

# 100x intervalls
ax.add_line(mlines.Line2D([0.01,10], [1,1000], color='green', ls='--', lw=0.5))
ax.add_line(mlines.Line2D([1,1000], [0.01,10], color='green', ls='--', lw=0.5))
plt.show()

In [ ]:
k = max(df['k'])

viridis = cm.get_cmap('turbo', k)
newcolors = viridis(np.linspace(0, 1, k))
pink = np.array([248/256, 24/256, 148/256, 1])
newcolors[:1, :] = pink
newcmp = ListedColormap(newcolors)

fig, ax = plt.subplots(figsize=(8,4))

im = ax.scatter(df['n'], df['dfvs_busy_cycle_no_clean'], c=df['k'], cmap=newcmp)
ax.set_yscale('log')
ax.set_xlabel('number of vertices')
ax.set_ylabel('time in seconds')

color_bar = fig.colorbar(im, orientation="vertical")
color_bar.ax.set_ylabel('k')
plt.savefig("all.png")
plt.show()